# Nihad's clustering neighborhoods Project

## First task - webscraping and co.

Importing libraries:

In [78]:
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.preprocessing import LabelEncoder
!conda install -c conda-forge lxml --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip3 install lxml

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



Setting the source Wikipedia's url value and calling the HTTP GET request on the url. Answer in the form of HTTP Response is saved into variable result. HTML code in textual representation can be found in attribute text:

In [79]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
result = requests.get(url)
#result.text

Attribute text contains unformatted clear text which is unusable without knowledge of HTML. We know that our results of interest are located in the HTML object table, so webscraping is done by selecting the HTML code between opening and closing tag of the table. At the end of the cell is used method read_html() of pandas to get the content of the table in the pandas dataframe: 

In [80]:
idx1 = result.text.find('<table class="wikitable sortable"')
idx2 = result.text.find('</table>',idx1)
str = result.text[idx1:idx2+8]
dl = pd.read_html(str)
df_raw = dl[0]

Because original column name of the third column on the Wikipedia page is different than the name from the task, it will be renamed from 'Neighbourhood' to 'Neighborhood' 

In [81]:
df_raw.columns.values[2]='Neighborhood'

Filtering of boroughs with 'Not assigned' value. Before and after the filtering operation the number of cells with those values of the column 'Borough' will be printed out: 

In [82]:
print('Control of not assigned boroughs before operation:',np.sum(df_raw['Borough']=='Not assigned'))
df_assigned = df_raw[df_raw['Borough']!='Not assigned']
df_assigned.reset_index(drop=True,inplace=True)
print('Control of not assigned boroughs after operation:',np.sum(df_assigned['Borough']=='Not assigned'))

Control of not assigned boroughs before operation: 77
Control of not assigned boroughs after operation: 0


Processing of neighborhoods with 'Not assigned' value. Before and after the assignment operation the number of cells with those values of the column 'Neighborhood' will be printed out: 

In [83]:
print('Control of not assigned neighborhoods before operation:',np.sum(df_assigned['Neighborhood']=='Not assigned'))
for i in range(0,df_assigned.shape[0]):
    if(df_assigned.iloc[i]['Neighborhood']=='Not assigned'):
        df_assigned.iloc[i]['Neighborhood'] = df_assigned.iloc[i]['Borough']
print('Control of not assigned neighborhoods after operation:',np.sum(df_assigned['Neighborhood']=='Not assigned'))

Control of not assigned neighborhoods before operation: 1
Control of not assigned neighborhoods after operation: 0


Processing of duplicate Postcodes

In [84]:
num_duplicates = df_assigned.shape[0] - df_assigned['Postcode'].unique().shape[0]
print('Number of duplicate values of postcodes before operation:',num_duplicates)
df_nonduplicated = ~df_assigned.duplicated(subset='Postcode',keep='first')
for i in range(0,df_assigned.shape[0]):
    if(df_assigned[df_assigned['Postcode'] == df_assigned.iloc[i]['Postcode']].shape[0] > 1):
        pCode = df_assigned.iloc[i]['Postcode']
        df_duplicates = df_assigned[df_assigned['Postcode'] == df_assigned.iloc[i]['Postcode']]
        for j in range(1,df_duplicates.shape[0]):
            df_assigned.iloc[i]['Neighborhood'] = df_assigned.iloc[i]['Neighborhood'] + ', ' + df_duplicates.iloc[j]['Neighborhood']
df_final = df_assigned[df_nonduplicated]
df_final.reset_index(drop=True, inplace=True)
num_duplicates = df_final.shape[0] - df_final['Postcode'].unique().shape[0]
print('Number of duplicate values of postcodes after operation:',num_duplicates)

Number of duplicate values of postcodes before operation: 108
Number of duplicate values of postcodes after operation: 0


For the insight purposes we display the final dataframe:

In [46]:
df_final.head()

At the end, the size of dataframe for the sake of control:

In [85]:
df_final.shape

(103, 3)

## Second task

Code which tries to call geocoder API, but without success:

In [48]:
#!pip install geocoder
#import geocoder # import geocoder

# define the dataframe columns
#column_names = ['Postcode','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
#df_final_geo = pd.DataFrame(columns=column_names)

#for i in range(0,df_final.shape[0]):
    
    # initialize your variable to None
#    lat_lng_coords = None
#    postal_code = df_final.iloc[i]['Postcode']
#    borough = df_final.iloc[i]['Borough']
#    neighborhood = df_final.iloc[i]['Neighborhood']
#    print(postal_code)
#    print('______start__________')
    # loop until you get the coordinates
#    j=1
#    while(lat_lng_coords is None):
#        print('versuch:',j)
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#        lat_lng_coords = g.latlng
#        j=j+1
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]
#    df_final_geo = df_final_geo.append({'Postcode': postal_code, 
#                                         'Borough': borough, 
#                                         'Neighborhood': neighborhood, 
#                                         'Latitude': Latitude, 
#                                         'Longitude': Longitude},ignore_index = True)
#    print(postal_code)
#    print('______end__________')

#df_final_geo

We could be happy that alternative exists:

In [86]:
#Alternative - because geocoder doesn't work
# define the dataframe columns
column_names = ['Postcode','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
df_final_geo = pd.DataFrame(columns=column_names)
df_latlong = pd.read_csv('http://cocl.us/Geospatial_data');
for i in range(0,df_final.shape[0]):
    postcode = df_final.iloc[i]['Postcode']
    borough = df_final.iloc[i]['Borough']
    neighborhood = df_final.iloc[i]['Neighborhood']
    latitude = df_latlong[df_latlong['Postal Code']==postcode].iloc[0]['Latitude']
    longitude = df_latlong[df_latlong['Postal Code']==postcode].iloc[0]['Longitude']
    #print('Pc:{},Lat:{},Lon:{}'.format(postcode,latitude,longitude))
    df_final_geo = df_final_geo.append({'Postcode': postcode, 
                                         'Borough': borough, 
                                         'Neighborhood': neighborhood, 
                                         'Latitude': latitude, 
                                         'Longitude': longitude},ignore_index = True)
df_final_geo.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## Third task

Visualising dataset with geolocation data on folium map

In [96]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final_geo['Latitude'], df_final_geo['Longitude'], df_final_geo['Borough'], df_final_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Foursquare API credentials

In [97]:
CLIENT_ID = 'W4FWDOZARKBV1SZXM1M2AJAFLODJANP4XTV121T5ASZ5WQ25' # your Foursquare ID
CLIENT_SECRET = 'KU150VOJ0ZSSGCLPCF4M05ATJQ4TJJQAIWVXLWBHS42L5ZPR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Define a function that will extract category name of the venue:

In [99]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Define a function that will get most frequent venue category around a given location and in specified radius:

In [101]:
def getTopVenue(latitude, longitude, radius, limit):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, limit)
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    #print(venues)
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    
    a = nearby_venues['categories'].value_counts()
    return a.index[0]

Choose boroughs of interest (for the analysis chosen are the boroughs that have Toronto in the name):

In [102]:
df_final_geo_toronto = df_final_geo[df_final_geo.Borough.str.contains('Toronto')]
print('New size of chosen set:',df_final_geo_toronto.shape)

New size of chosen set: (38, 5)


For the purpose of clustering, the most frequent category of the venue in neighborhood is determined and added to dataset.   
Find the most frequent venues and prepare dataset for clustering:

In [103]:
column_names = ['Postcode','Borough', 'Neighborhood', 'Latitude', 'Longitude','TopVenue'] 

# instantiate the dataframe
df_toronto = pd.DataFrame(columns=column_names)

for i in range(0,df_final_geo_toronto.shape[0]):
    neighborhood_postcode = df_final_geo_toronto.iloc[i]['Postcode']
    borough = df_final_geo_toronto.iloc[i]['Borough']
    neighborhood_latitude = df_final_geo_toronto.iloc[i]['Latitude'] # neighborhood latitude value
    neighborhood_longitude = df_final_geo_toronto.iloc[i]['Longitude'] # neighborhood longitude value
    neighborhood_name = df_final_geo_toronto.iloc[i]['Neighborhood'] # neighborhood name
    
    radius=500
    limit=100
    top_venue = getTopVenue(neighborhood_latitude, neighborhood_longitude, radius, limit)

    df_toronto = df_toronto.append({'Postcode': neighborhood_postcode,
                       'Borough': borough,
                       'Neighborhood': neighborhood_name,
                       'Latitude': neighborhood_latitude,
                       'Longitude': neighborhood_longitude,
                       'TopVenue': top_venue},ignore_index = True)
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,TopVenue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,Coffee Shop
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,Clothing Store
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Coffee Shop
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Health Food Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Coffee Shop


Encode the labels with non-numeric values for clustering:

In [104]:
#label encoding
le_postcode = LabelEncoder()
le_borough = LabelEncoder()
le_venue = LabelEncoder()
le_postcode.fit(df_toronto['Postcode'])
df_toronto['Postcode'] = le_postcode.transform(df_toronto['Postcode'])
le_borough.fit(df_toronto['Borough'])
df_toronto['Borough'] = le_borough.transform(df_toronto['Borough'])
le_venue.fit(df_toronto['TopVenue'])
df_toronto['TopVenue'] = le_venue.transform(df_toronto['TopVenue'])
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,TopVenue
0,13,1,"Harbourfront, Regent Park",43.654260,-79.360636,4
1,14,1,"Ryerson, Garden District",43.657162,-79.378937,3
2,15,1,St. James Town,43.651494,-79.375418,4
3,0,2,The Beaches,43.676357,-79.293031,9
4,16,1,Berczy Park,43.644771,-79.373306,4


Clustering with K-Means:

In [105]:
# set number of clusters
kclusters = 5
from sklearn.cluster import KMeans
clustering_data = df_toronto.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering_data)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
df_toronto['Cluster labels'] = kmeans.labels_
df_toronto['Borough'] = le_borough.inverse_transform(df_toronto['Borough'])
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,TopVenue,Cluster labels
0,13,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,4,3
1,14,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,3,3
2,15,Downtown Toronto,St. James Town,43.651494,-79.375418,4,3
3,0,East Toronto,The Beaches,43.676357,-79.293031,9,4
4,16,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,3


Visualize clusters

In [106]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_cluster['Latitude'], df_cluster['Longitude'], df_cluster['Borough'], df_cluster['Cluster labels']):
    label = folium.Popup(poi + ' Cluster ' + np.str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters